In [8]:
import sys, os, csv
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import rdkit.Chem.rdFMCS as MCS

In [9]:
import py3Dmol
from ipywidgets import interact, interactive, fixed

def drawit(m,p,confId=-1):
        mb = Chem.MolToMolBlock(m,confId=confId)
        p.removeAllModels()
        p.addModel(mb,'sdf')
        p.setStyle({'stick':{}})
        p.setBackgroundColor('0xeeeeee')
        p.zoomTo()
        return p.show()

In [10]:
#load UAA smile strings
smi_db = {}
lines = open("UAA-info.csv", 'r').readlines()
for l in lines[:]:
    es = l.split(' ')
    ndx = es[0][:-1]
    smi = es[1].strip()
    #print ndx, smi
    smi_db[ndx] = smi
smi_db

{'ONB-D': 'N[C@H](CC(OCC1=C([N+]([O-])=O)C=CC=C1)=O)C(O)=O',
 'BnO-D': 'N[C@H](CC(OCC1=CC=CC=C1)=O)C(O)=O'}

In [11]:
#define reaction
rxn_smarts = "[C:1](=[O:2])[OH:3].[OH:4][PH0:5][OH:6]>>[C:1](=[O:2])[OH0:4][P:5][O-:6]"
rxn = AllChem.ReactionFromSmarts(rxn_smarts)

In [12]:
#load AMP template
suppl = Chem.SDMolSupplier("AMP.sdf")
for mol in suppl:
    AMP_tmpl = mol
suppl = Chem.SDMolSupplier("stub-D.sdf")
for mol in suppl:
    stubD = mol


In [15]:
pv = py3Dmol.view(width=400,height=400)
interact(drawit, m=fixed(stubD),p=fixed(pv),confId=(0,AMP_tmpl.GetNumConformers()-1));

interactive(children=(IntSlider(value=0, description='confId', max=0), Output()), _dom_classes=('widget-intera…

In [17]:
import numpy
def CalcConfRMS(N, conf1, conf2):
    ssr = 0                                                                        
    for i in range(N):                                             
        d = conf1.GetAtomPosition(i).Distance(conf2.GetAtomPosition(i))              
        ssr += d * d                                                                 
    ssr /= N
    return numpy.sqrt(ssr)

In [18]:
def GenMutiConfMol(mol, tmpl, num_iter, rms_cut, w):
    #constrained and align
    res = MCS.FindMCS([mol, tmpl], threshold=0.9, completeRingsOnly=True)
    p = Chem.MolFromSmarts(res.smartsString)
    core = AllChem.DeleteSubstructs(AllChem.ReplaceSidechains(tmpl,p), Chem.MolFromSmiles('*'))
    core.UpdatePropertyCache()
    print(Chem.MolToSmiles(core))
    #generate confs
    conformers = None
    last_iter = 0
    for i in range(num_iter):
        AllChem.ConstrainedEmbed(mol, core, randomseed=i+111)
        ebrms = float(mol.GetProp('EmbedRMS'))
        if ebrms<0.10:
            #check 
            if conformers == None:
                #save the first one
                conformers = Chem.MolFromMolBlock( Chem.MolToMolBlock(mol) )
                Chem.SanitizeMol(conformers)
                conformers = Chem.AddHs(conformers)
                conformers.AddConformer(mol.GetConformer(0))
                w.write(mol)
            else:
                confs = conformers.GetConformers()
                rmslst = [ CalcConfRMS(mol.GetNumAtoms(), conf, mol.GetConformer(0)) for conf in confs ]
                minrms = min(rmslst)
                if minrms > rms_cut:
                    #save if not close any old rotamer
                    conformers.AddConformer(mol.GetConformer(0))
                    w.write(mol)
                    print(i, 'RMS:', minrms)
                    last_iter = i
                else:
                    if i-last_iter>100:
                        print("Warning: early quit at iter=", i)
                        break
    return conformers

In [14]:
for ndx in smi_db.keys():
    k = ndx
    t = ndx[4]
    smi = smi_db[k]
    #init
    d = "U"+str(k)
    if not os.path.isdir(d):
        os.mkdir(d)
    #create
    print(k, smi)
    UAA = AllChem.MolFromSmiles(smi) #Chem.MolToSmiles(mol, isomericSmiles=True)
    #react
    products = rxn.RunReactants((UAA, AMP_tmpl))
    print(len(products))
    writer = Chem.SDWriter(d+"/confs.sdf")
    for prod in products[1:]:
        #fix prod
        product = prod[0]
        Chem.SanitizeMol(product)
        #product = Chem.RemoveHs(product)
        product = Chem.AddHs(product)
        if t == "D": stub = stubD
        GenMutiConfMol(product, stub, 5000, 0.5, writer)
    writer.close()

ONB-D N[C@H](CC(OCC1=C([N+]([O-])=O)C=CC=C1)=O)C(O)=O
2
NCC(=O)OP(=O)(O)OCC1OC(n2cnc3c(N)ncnc32)C(O)C1O
16 RMS: 4.3255220966121
18 RMS: 4.536540290576246
20 RMS: 1.886567067992444
21 RMS: 3.636478053036757
24 RMS: 2.31821277472175
25 RMS: 0.5399387200014606
26 RMS: 1.8466347864241104
30 RMS: 2.5068770518519616
42 RMS: 1.2907469612629265
50 RMS: 2.9482109876443707
61 RMS: 1.8298901004674046
66 RMS: 1.6414936128759696
71 RMS: 1.7584648858872258
72 RMS: 1.672516647042382
76 RMS: 1.2055147903552337
77 RMS: 0.7398873790302197
79 RMS: 2.2426199856000233
88 RMS: 0.5050506541303272
89 RMS: 0.5542130703187006
90 RMS: 0.6794264093846832
91 RMS: 1.7590503671287647
92 RMS: 0.6036356678399913
94 RMS: 2.114989196921169
95 RMS: 0.7150846165706102
98 RMS: 1.7234851637444097
101 RMS: 1.1783579791392371
104 RMS: 2.814093157826742
105 RMS: 0.658299135384715
107 RMS: 1.833364352154719
111 RMS: 1.365495881935624
115 RMS: 1.5321413538028434
117 RMS: 2.7564882062355838
119 RMS: 1.9253918218732309
124 RMS: 0.

1459 RMS: 0.5984764233463542
1464 RMS: 0.965108918673048
1473 RMS: 0.6770943212137209
1474 RMS: 0.5974117855459221
1476 RMS: 0.5544943623171504
1478 RMS: 0.6213982469886276
1489 RMS: 0.7269712337019721
1497 RMS: 1.4156561312723221
1502 RMS: 0.5428934365594082
1509 RMS: 0.600887923765993
1526 RMS: 0.611155774713468
1528 RMS: 0.5351119084983752
1530 RMS: 0.6366356074977366
1538 RMS: 0.5293687226011314
1540 RMS: 0.5001008694447212
1541 RMS: 0.6638967307875129
1542 RMS: 0.900298138256686
1546 RMS: 0.563539233550033
1548 RMS: 0.6582673237192589
1549 RMS: 0.5722673168897051
1565 RMS: 0.6008598872044053
1585 RMS: 0.6243624273704541
1588 RMS: 0.5603264518074789
1589 RMS: 0.7521015661990111
1595 RMS: 0.5544742177589859
1621 RMS: 0.5897524820524985
1630 RMS: 0.6360468069626816
1631 RMS: 1.1407220682980694
1640 RMS: 0.5555391572643668
1645 RMS: 0.7247080481218449
1647 RMS: 0.6183395756220773
1648 RMS: 0.7165204793940496
1655 RMS: 0.685813077264542
1660 RMS: 0.5734860920019766
1661 RMS: 0.68108778

3581 RMS: 0.5018691077476167
3584 RMS: 0.6984485010138963
3593 RMS: 0.5005939522355882
3604 RMS: 1.0738758449224592
3605 RMS: 0.571613632408476
3607 RMS: 0.5363374851013779
3628 RMS: 0.6393521060123983
3629 RMS: 0.5731902057095947
3630 RMS: 0.5673343802776688
3635 RMS: 0.6475392024480312
3683 RMS: 0.6512805068641289
3688 RMS: 0.5295073635640823
3692 RMS: 0.5429259339351259
3693 RMS: 0.5042325528503065
3696 RMS: 0.8224569343863092
3698 RMS: 0.6253220312270801
3703 RMS: 0.5092077514087822
3718 RMS: 0.5753972231122052
3740 RMS: 0.5267161366587786
3748 RMS: 0.7627401284352088
3759 RMS: 0.6422069514649247
3765 RMS: 0.7050554440109033
3768 RMS: 0.7845670936021427
3769 RMS: 0.6730066718149574
3778 RMS: 0.5394729036682366
3787 RMS: 0.507992704219403
3791 RMS: 0.5228939684438512
3807 RMS: 0.5504763003095624
3820 RMS: 0.6073825600442142
3821 RMS: 0.5034341891863794
3824 RMS: 0.5440452014965885
3826 RMS: 0.594618495645106
3836 RMS: 0.61378971472037
3851 RMS: 0.5692062690225389
3868 RMS: 0.5727523

954 RMS: 1.3104976432781819
973 RMS: 0.5139391788147811
983 RMS: 0.5663353184040991
1005 RMS: 0.5217462367571292
1021 RMS: 0.5467181194765193
1062 RMS: 0.5039307428392673
1089 RMS: 0.6212444217421602
1095 RMS: 0.6227908006211673
1121 RMS: 0.5485601235838624
1126 RMS: 0.5569360333285107
1129 RMS: 0.6066827356023695
1137 RMS: 0.6099120227649475
1148 RMS: 0.5019599582813212
1150 RMS: 0.5025810295858375
1156 RMS: 0.6173157062169908
1161 RMS: 0.5059153319544573
1183 RMS: 0.5311688640016049
1186 RMS: 0.5940823030434105
1191 RMS: 0.5988995225287295
1203 RMS: 0.5296829575420348
1211 RMS: 0.5567646882453882
1217 RMS: 0.5044659412374909
1235 RMS: 0.5248101235064957
1266 RMS: 0.5227099089001339
1323 RMS: 0.5853592057760422
1325 RMS: 0.6001059511385439
1326 RMS: 0.5879510155548875
1335 RMS: 0.9999917374759253
1352 RMS: 0.5837245969344357
1402 RMS: 0.5861044643416732
1414 RMS: 0.5458826668994993
1422 RMS: 0.5037587377828737
1439 RMS: 0.5632067416868627
1455 RMS: 0.6109521287857334
1460 RMS: 0.54494

335 RMS: 0.6775702254922058
336 RMS: 0.5494391552300382
337 RMS: 0.8547803752414523
342 RMS: 1.2923762946031787
344 RMS: 0.7957163124321905
348 RMS: 0.852759985968255
350 RMS: 2.0408384971650757
351 RMS: 1.6436817408251552
352 RMS: 1.0467442309881037
354 RMS: 0.6353316540948174
360 RMS: 0.6736066424055449
365 RMS: 0.5369777675618675
366 RMS: 0.8680201570297021
367 RMS: 1.578399117159075
368 RMS: 0.5615407458766036
372 RMS: 0.726862955514157
373 RMS: 1.4946481915335372
375 RMS: 1.4106501274023844
376 RMS: 0.7088448564629112
379 RMS: 1.1855223387035008
387 RMS: 0.7121968510420759
389 RMS: 0.5656589179444151
392 RMS: 1.17341447558676
398 RMS: 0.5788168143344958
405 RMS: 1.2251089174857792
406 RMS: 1.7149868592413586
412 RMS: 0.5740701268189966
413 RMS: 1.7621222799107643
418 RMS: 1.7432533433278496
420 RMS: 0.7122056202501559
425 RMS: 0.8788128088479247
426 RMS: 1.3237058420276688
435 RMS: 0.5872431944374774
440 RMS: 0.7065367876656751
447 RMS: 0.8585074079635189
449 RMS: 0.60771436915551

1510 RMS: 0.6044266247683345
1517 RMS: 0.6209884578680657
1521 RMS: 0.5630078960822398
1523 RMS: 0.535978459322515
1525 RMS: 1.3462445687092437
1535 RMS: 1.3646858025460922
1536 RMS: 0.7100178885595014
1538 RMS: 0.634136201764496
1539 RMS: 0.5401712779896488
1545 RMS: 1.3968117725452966
1547 RMS: 0.6998546416530933
1549 RMS: 0.7581124454727621
1556 RMS: 1.172518116244011
1559 RMS: 0.6059367054323577
1565 RMS: 0.6402121422488455
1580 RMS: 0.5735829233298979
1581 RMS: 0.6977216464845225
1590 RMS: 1.0192395169554085
1596 RMS: 0.9321713009448054
1600 RMS: 1.0108510762310836
1607 RMS: 0.6462790244532712
1620 RMS: 1.2344493016387055
1622 RMS: 0.7327760024075851
1626 RMS: 0.6134104547153657
1627 RMS: 1.3741108699597604
1628 RMS: 0.9020916629866317
1632 RMS: 0.7837998164133179
1633 RMS: 0.5105091457780601
1639 RMS: 0.6031816332151301
1646 RMS: 0.6335346153152588
1659 RMS: 0.8247757171967756
1665 RMS: 0.8417818717589991
1688 RMS: 0.692885729931625
1694 RMS: 0.6336850988638653
1695 RMS: 0.550341

2978 RMS: 0.6445063834536964
2981 RMS: 0.5243299951743864
2983 RMS: 0.544880739455478
2987 RMS: 0.6919025949606795
3003 RMS: 0.8824486154957769
3007 RMS: 0.6100506140854142
3008 RMS: 0.5687648770217478
3009 RMS: 0.8640388298176108
3016 RMS: 0.5856891822267549
3018 RMS: 1.4928454995528664
3027 RMS: 0.669723626005634
3028 RMS: 0.7680031206443206
3043 RMS: 0.7201938717008586
3047 RMS: 0.660201235303303
3048 RMS: 1.0671799597896119
3052 RMS: 0.6068205823550444
3053 RMS: 0.5517291766684579
3059 RMS: 0.693629583841103
3063 RMS: 0.7515968486854135
3065 RMS: 0.6325444329202344
3071 RMS: 0.5613624493915973
3074 RMS: 0.5114035758597543
3075 RMS: 0.5328090538655317
3078 RMS: 0.5216061700344471
3080 RMS: 0.8118380965209577
3082 RMS: 0.6156386136302223
3086 RMS: 0.5720474132689053
3088 RMS: 0.8424222888127174
3091 RMS: 0.6178962487075789
3095 RMS: 1.3294585564232568
3111 RMS: 0.656146660709662
3116 RMS: 1.093699690510275
3119 RMS: 0.6558031320590858
3120 RMS: 0.5741052691895816
3134 RMS: 0.52632923

4466 RMS: 0.7645593785075334
4470 RMS: 0.6342342946060839
4479 RMS: 0.9112765671024436
4482 RMS: 0.6057993150653157
4488 RMS: 0.6418838596809695
4498 RMS: 0.6297674898402089
4502 RMS: 0.565717275735582
4506 RMS: 0.790843834426853
4510 RMS: 0.6519601347189133
4513 RMS: 0.5106292900455267
4527 RMS: 0.6062506594304193
4528 RMS: 0.5721492405435533
4536 RMS: 0.5809548464012843
4550 RMS: 0.560144905602635
4556 RMS: 0.6647555111490621
4559 RMS: 0.5640354384461198
4571 RMS: 0.6291957931543407
4573 RMS: 0.5147365412718324
4609 RMS: 0.7613776812047562
4615 RMS: 0.7258938717261575
4628 RMS: 0.6279548919947795
4630 RMS: 0.5125833650262409
4636 RMS: 0.532926192776582
4646 RMS: 0.8087846687348508
4673 RMS: 0.5273424011378574
4681 RMS: 0.5913765560680331
4684 RMS: 0.728187646516378
4688 RMS: 0.5822533099077007
4699 RMS: 0.6553681465246258
4700 RMS: 0.5361629573978758
4709 RMS: 0.848878368997758
4711 RMS: 0.5124073722936473
4714 RMS: 0.8065494326517461
4722 RMS: 0.5991710394137111
4734 RMS: 0.67891801

819 RMS: 0.6505146381227441
823 RMS: 0.6137103136481724
831 RMS: 1.7612451307672836
833 RMS: 0.5177871823854774
841 RMS: 0.6018857832886425
846 RMS: 0.6731110459730563
851 RMS: 1.3621224501495899
864 RMS: 0.5740670513202722
866 RMS: 0.5860031824893783
870 RMS: 1.3766570463987224
872 RMS: 0.5902383112076963
878 RMS: 1.2048713268680369
890 RMS: 0.6493562798303059
896 RMS: 0.5842806293071716
899 RMS: 0.5978939047633463
901 RMS: 0.6731826501513051
902 RMS: 0.7031749321925537
904 RMS: 0.5790611889298323
909 RMS: 0.8162929030414328
911 RMS: 1.2183082578910387
914 RMS: 0.5844316702465856
917 RMS: 0.5476772835543349
921 RMS: 0.5181557109013113
926 RMS: 1.4065036198592786
935 RMS: 0.6206897378840317
953 RMS: 0.6540970817980812
956 RMS: 0.5241704362285259
961 RMS: 0.6060284392593946
966 RMS: 1.0779347667767107
970 RMS: 0.5436313192877766
971 RMS: 1.6000102267167227
979 RMS: 0.6246669019611476
989 RMS: 1.153230611424885
992 RMS: 0.5970448872368309
1003 RMS: 1.3975089758806758
1006 RMS: 0.57071825

2734 RMS: 0.6217824719709327
2735 RMS: 0.5303462274688797
2736 RMS: 0.6564125204962165
2764 RMS: 0.507514131600994
2768 RMS: 0.5204636058985184
2786 RMS: 0.6075408237919258
2788 RMS: 0.5674312715273127
2809 RMS: 0.6247871965537036
2814 RMS: 0.5426459014571182
2820 RMS: 0.5142778791273432
2824 RMS: 0.521203774198086
2831 RMS: 0.590025084202328
2855 RMS: 0.671246652321875
2860 RMS: 0.5330594700440564
2861 RMS: 0.6512424806851558
2874 RMS: 0.7977685311096843
2877 RMS: 0.5091464359852095
2878 RMS: 0.6161255357866227
2887 RMS: 1.209248322206596
2895 RMS: 0.6924159274066028
2904 RMS: 0.5239736858420203
2913 RMS: 0.5312913013379551
2919 RMS: 0.616260644367083
2921 RMS: 0.6223182460946057
2951 RMS: 0.5799458464777493
2962 RMS: 0.5694781562520579
2965 RMS: 0.5475379279092703
2969 RMS: 0.7598864655947101
3008 RMS: 0.5003304730174163
3023 RMS: 0.5333899009936651
3044 RMS: 0.616831484271156
3058 RMS: 0.5390929151827756
3068 RMS: 0.5866014722459321
3069 RMS: 0.5406958908035211
3082 RMS: 0.656352788